In [1]:
import gwaslab as gl

In [2]:
mysumstats = gl.Sumstats(
    "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/21001_T2T_liftover_rsid.tsv.gz",
    snpid="SNPID",
    chrom="CHR",
    pos="POS",
    ea="EA",            # In your data, it's now EA (was ALT)
    nea="NEA",          # In your data, it's now NEA (was REF)
    eaf="EAF",          # In your data, it's now EAF (was minor_AF/Frq)
    beta="BETA",
    se="SE",
    p="P",
    # Note: 'Dir' and 'N' are not in your column list, 
    # so we should omit them or point to the correct ones.
    # n="n_complete_samples", # Optional: if you want to use sample size
    verbose=True,
    readargs={'sep': '\t'}
)

2026/01/10 12:33:12 GWASLab v4.0.4 https://cloufield.github.io/gwaslab/
2026/01/10 12:33:12 (C) 2022-2026, Yunye He, Kamatani Lab, GPL-3.0 license, gwaslab@gmail.com
2026/01/10 12:33:12 Python version: 3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]
2026/01/10 12:33:13 Start to initialize gl.Sumstats from file :/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/21001_T2T_liftover_rsid.tsv.gz
2026/01/10 12:33:37  -Reading columns          : P,BETA,POS,CHR,NEA,EAF,EA,SNPID,SE
2026/01/10 12:33:37  -Renaming columns to      : P,BETA,POS,CHR,NEA,EAF,EA,SNPID,SE
2026/01/10 12:33:37  -Current Dataframe shape : 13640038  x  9
2026/01/10 12:33:37  -Initiating a status column: STATUS ...
2026/01/10 12:33:37 #WARNING! Version of genomic coordinates is unknown for homo sapiens (build: 99)...
2026/01/10 12:33:40 Start to reorder the columns ...(v4.0.4)
2026/01/10 12:33:40  -Reordering columns to    : SNPID,CHR,POS,EA,NEA,STATUS,EAF,BETA,SE,P
2026/01/10 12:33:41 Finished reordering the columns.
2026/01/10 1

In [3]:
mysumstats.data

,SNPID,CHR,POS,EA,NEA,STATUS,EAF,BETA,SE,P
0,1:692794:CA:C,1,119016,C,CA,9999999,0.110590,-0.016436,0.019585,0.401342
1,1:693731:A:G,1,119953,G,A,9999999,0.115767,-0.004255,0.018507,0.818155
2,1:707522:G:C,1,133743,C,G,9999999,0.097255,-0.010428,0.020804,0.616190
3,1:717587:G:A,1,144100,A,G,9999999,0.015679,0.001174,0.049643,0.981132
4,1:723329:A:T,1,149839,T,A,9999999,0.001732,-0.117102,0.146483,0.424044
...,...,...,...,...,...,...,...,...,...,...
13640033,X:154929412:C:T,23,153165819,T,C,9999999,0.245473,-0.016260,0.010723,0.129427
13640034,X:154929637:CT:C,23,153166044,C,CT,9999999,0.229726,-0.027098,0.011190,0.015456
13640035,X:154929952:CAA:C,23,153166359,C,CAA,9999999,0.239430,-0.020494,0.011278,0.069202
13640036,X:154930230:A:G,23,153166637,G,A,9999999,0.245887,-0.016347,0.010721,0.127334


In [5]:
# import gwaslab as gl

# GWASLab has a built-in helper to fetch some of these
# This will download the auto-chromosome dbSNP reference for hg38 which includes frequencies
gl.download_ref("1kg_dbsnp151_hg38_auto")

2026/01/10 12:41:01 Start to download  1kg_dbsnp151_hg38_auto  ...
2026/01/10 12:41:01  -Downloading to: /home/ofgeha/.gwaslab/1kg_dbsnp151_hg38_auto.txt.gz
2026/01/10 12:41:01  -File /home/ofgeha/.gwaslab/1kg_dbsnp151_hg38_auto.txt.gz exists.
2026/01/10 12:41:01  - Updating record in config file...
2026/01/10 12:41:01 Downloaded  1kg_dbsnp151_hg38_auto  successfully!


In [ ]:
# Path to the FASTA you just successfully downloaded
ref_fasta = "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/hg38.fa"

# Path to the VCF (Update filename if you used gl.download_ref)
ref_vcf = "/home/ofgeha/.gwaslab/1kg_dbsnp151_hg38_auto.txt.gz"

mysumstats.harmonize(
    ref_seq=ref_fasta,
    ref_infer=ref_vcf,
    ref_alt_freq="EUR_AF", 
    basic_check=True,
    threads=4
)

2026/01/10 12:55:22 Start to check SNPID/rsID ...(v4.0.4)
2026/01/10 12:55:22  -Checking SNPID data type...
2026/01/10 12:55:22  -Checking if SNPID contains NA strings :na,NA,Na,Nan,NaN,<NA>,null,NULL,#N/A,#VALUE!,N/A,n/a,missing,...
2026/01/10 12:55:22  -Checking if SNPID is CHR:POS:NEA:EA...(separator: - ,: , _)
2026/01/10 12:55:32 Finished checking SNPID/rsID.
2026/01/10 12:55:32 Start to fix chromosome notation (CHR) ...(v4.0.4)
2026/01/10 12:55:32  -Checking CHR data type...
2026/01/10 12:55:36  -Variants with standardized chromosome notation: 13640038
2026/01/10 12:55:36  -All CHR are already fixed...
2026/01/10 12:55:39 Finished fixing chromosome notation (CHR).
2026/01/10 12:55:39 Start to fix basepair positions (POS) ...(v4.0.4)
2026/01/10 12:55:39  -Trying to convert datatype for POS: Int64 -> Int64...
2026/01/10 12:55:40  -Position bound:(0 , 250,000,000)
2026/01/10 12:55:40  -No outlier variants were removed.
2026/01/10 12:55:41 Finished fixing basepair positions (POS).
202

,SNPID,CHR,POS,EA,NEA,STATUS,EAF,BETA,SE,P
0,1:692794:CA:C,1,119016,C,CA,9960399,0.110590,-0.016436,0.019585,0.401342
1,1:693731:A:G,1,119953,G,A,9960090,0.115767,-0.004255,0.018507,0.818155
2,1:707522:G:C,1,133743,C,G,9960099,0.097255,-0.010428,0.020804,0.616190
3,1:717587:G:A,1,144100,A,G,9960090,0.015679,0.001174,0.049643,0.981132
4,1:723329:A:T,1,149839,T,A,9960099,0.001732,-0.117102,0.146483,0.424044
...,...,...,...,...,...,...,...,...,...,...
13640033,X:154929412:C:T,23,153165819,T,C,9960090,0.245473,-0.016260,0.010723,0.129427
13640034,X:154929637:CT:C,23,153166044,C,CT,9960399,0.229726,-0.027098,0.011190,0.015456
13640035,X:154929952:CAA:C,23,153166359,C,CAA,9960399,0.239430,-0.020494,0.011278,0.069202
13640036,X:154930230:A:G,23,153166637,G,A,9960090,0.245887,-0.016347,0.010721,0.127334


In [9]:
mysumstats.data

,SNPID,CHR,POS,EA,NEA,STATUS,EAF,BETA,SE,P
0,1:692794:CA:C,1,119016,C,CA,9960399,0.110590,-0.016436,0.019585,0.401342
1,1:693731:A:G,1,119953,G,A,9960090,0.115767,-0.004255,0.018507,0.818155
2,1:707522:G:C,1,133743,C,G,9960099,0.097255,-0.010428,0.020804,0.616190
3,1:717587:G:A,1,144100,A,G,9960090,0.015679,0.001174,0.049643,0.981132
4,1:723329:A:T,1,149839,T,A,9960099,0.001732,-0.117102,0.146483,0.424044
...,...,...,...,...,...,...,...,...,...,...
13640033,X:154929412:C:T,23,153165819,T,C,9960090,0.245473,-0.016260,0.010723,0.129427
13640034,X:154929637:CT:C,23,153166044,C,CT,9960399,0.229726,-0.027098,0.011190,0.015456
13640035,X:154929952:CAA:C,23,153166359,C,CAA,9960399,0.239430,-0.020494,0.011278,0.069202
13640036,X:154930230:A:G,23,153166637,G,A,9960090,0.245887,-0.016347,0.010721,0.127334


### Check summary

In [10]:
mysumstats.summary()

{'overview': {'Row_num': '13640038',
  'Column_num': '10',
  'Column_names': 'SNPID,CHR,POS,EA,NEA,STATUS,EAF,BETA,SE,P',
  'Last_checked_time': 'Sat Jan 10 12:59:45 2026',
  'QC and Harmonization': {'basic_check': {'performed': True,
    'last_executed': '2026-01-10 12:56:19',
    'parameters_used': {'basic_check': True,
     'ref_seq': '/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/hg38.fa',
     'ref_rsid_tsv': None,
     'ref_rsid_vcf': None,
     'ref_infer': '/home/ofgeha/.gwaslab/1kg_dbsnp151_hg38_auto.txt.gz',
     'ref_alt_freq': 'EUR_AF',
     'ref_maf_threshold': 0.4,
     'maf_threshold': 0.4,
     'threads': 4,
     'remove': False,
     'check_ref_kwargs': {},
     'remove_dup_kwargs': {},
     'assign_rsid_kwargs': {},
     'infer_strand_kwargs': {},
     'flip_allele_stats_kwargs': {},
     'liftover_kwargs': {},
     'fix_id_kwargs': {},
     'fix_chr_kwargs': {},
     'fix_pos_kwargs': {},
     'fix_allele_kwargs': {},
     'sanity_check_stats_kwargs': {},
     'normalize_